In [ ]:
# 1. Import and Install Dependencies

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive',force_remount=True)

In [ ]:
#!pip install -q tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

# 1. Import dependencies

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Model Definition

In [ ]:
#%% (2) Model definition.
import tensorflow.compat.v1 as tf

class GRU:
    """Implementation of a Gated Recurrent Unit (GRU) as described in [1].
    
    [1] Chung, J., Gulcehre, C., Cho, K., & Bengio, Y. (2014). Empirical evaluation of gated recurrent neural networks on sequence modeling. arXiv preprint arXiv:1412.3555.
    
    Arguments
    ---------
    input_dimensions: int
        The size of the input vectors (x_t).
    hidden_size: int
        The size of the hidden layer vectors (h_t).
    dtype: obj
        The datatype used for the variables and constants (optional).
    """
    
    def __init__(self, input_dimensions, hidden_size, dtype=tf.float32):
        self.input_dimensions = input_dimensions
        self.hidden_size = hidden_size
        
        # Weights for input vectors of shape (input_dimensions, hidden_size)
        self.Wr = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wr')
        self.Wz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wz')
        self.Wh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.input_dimensions, self.hidden_size), mean=0, stddev=0.01), name='Wh')
        
        # Weights for hidden vectors of shape (hidden_size, hidden_size)
        self.Ur = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Ur')
        self.Uz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uz')
        self.Uh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size, self.hidden_size), mean=0, stddev=0.01), name='Uh')
        
        # Biases for hidden vectors of shape (hidden_size,)
        self.br = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='br')
        self.bz = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bz')
        self.bh = tf.Variable(tf.truncated_normal(dtype=dtype, shape=(self.hidden_size,), mean=0, stddev=0.01), name='bh')
        
        # Define the input layer placeholder
        self.input_layer = tf.placeholder(dtype=dtype, shape=(None, None, input_dimensions), name='input')
        
        # Put the time-dimension upfront for the scan operator
        self.x_t = tf.transpose(self.input_layer, [1, 0, 2], name='x_t')
        
        # A little hack (to obtain the same shape as the input matrix) to define the initial hidden state h_0
        self.h_0 = tf.matmul(self.x_t[0, :, :], tf.zeros(dtype=dtype, shape=(input_dimensions, hidden_size)), name='h_0')
        
        # Perform the scan operator
        self.h_t_transposed = tf.scan(self.forward_pass, self.x_t, initializer=self.h_0, name='h_t_transposed')
        
        # Transpose the result back
        self.h_t = tf.transpose(self.h_t_transposed, [1, 0, 2], name='h_t')

    def forward_pass(self, h_tm1, x_t):
        """Perform a forward pass.
        
        Arguments
        ---------
        h_tm1: np.matrix
            The hidden state at the previous timestep (h_{t-1}).
        x_t: np.matrix
            The input vector.
        """
        # Definitions of z_t and r_t
        
        r_t = tf.sigmoid(tf.matmul(x_t, self.Wr) + tf.matmul(h_tm1, self.Ur) + self.br)
        z_t = tf.sigmoid((tf.matmul(x_t, self.Wz) + tf.matmul(h_tm1, self.Uz))*r_t + self.bz)
        
        # Definition of h~_t
        h_proposal = tf.nn.elu(tf.matmul(x_t, self.Wh) + tf.matmul(tf.multiply(r_t, h_tm1), self.Uh) + self.bh)
        
        # Compute the next hidden state
        h_t = tf.multiply(1 - z_t, h_tm1) + tf.multiply(z_t, h_proposal)
        
        return h_t

# 4. Initialize and train the model.

In [ ]:
#%% (3) Initialize and train the model.
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
session = tf.Session()

# The input has 2 dimensions: dimension 0 is reserved for the first term and dimension 1 is reverved for the second term
input_dimensions = 1662 #features
classes=13
batch_size =50
hidden_size = 32
time_size = 30 #window size
lr = 1e-3
epochs = 101
# Initialize a session

# Create a new instance of the GRU model
gru = GRU(input_dimensions, hidden_size)

# Add an additional layer on top of each of the hidden state outputs
W_output = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=(hidden_size*time_size, classes), mean=0, stddev=0.05))
b_output = tf.Variable(tf.truncated_normal(dtype=tf.float32, shape=(classes,), mean=0, stddev=0.05))
output = tf.map_fn(lambda h_t: tf.nn.softsign(tf.matmul(tf.reshape(h_t,(1,hidden_size*time_size)), W_output) + b_output), gru.h_t)
output = tf.reshape(output,((batch_size, classes)))
# Create a placeholder for the expected output
expected_output = tf.placeholder(dtype=tf.float32, shape=(batch_size, classes), name='expected_output')

# Just use quadratic loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(expected_output,output))
print(expected_output,W_output,b_output,output,loss)
accs = tf.reduce_mean(tf.metrics.accuracy(labels=tf.argmax(expected_output, axis=1), predictions=tf.argmax(output,axis=1))) 
# loss = tf.reduce_sum(0.5 * tf.pow(output - expected_output, 2)) / float(batch_size)

# Use the Adam optimizer for training
train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

# Initialize all the variables
init_variables = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()

session.run(init_variables)
session.run(init_l)


In [ ]:
tf.compat.v1.disable_eager_execution()

saver = tf.compat.v1.train.Saver()
saved_path ='gru/gru-219' #saved model
with tf.compat.v1.Session() as sess:
    saver.restore(sess, saved_path)

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [ ]:
import pickle 
import time
#from google.colab.patches import cv2_imshow
from IPython.display import clear_output

le = pickle.load(open('labelencoder.pkl','rb')) #saved label encoder
#cap = cv2.VideoCapture('/content/ISL_CSLRT_Corpus/Videos_Sentence_Level/thank you so much/thank you so much (1).MP4')
cap = cv2.VideoCapture(0)

# Set mediapipe model 
window=time_size #defined above

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    count=0
    frames=[]
    while cap.isOpened():
        count+=1
        # Read feed
        ret, frame = cap.read()

        # Make detections
        if ret==True:
            start = time.time()
            image, results = mediapipe_detection(frame, holistic)
            print(results)
            pose = []
            for res in results.pose_landmarks.landmark:
                test = np.array([res.x, res.y, res.z, res.visibility])
                pose.append(test)
            pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
            face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
            lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
            rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
            result_test = extract_keypoints(results)
            frames.append(result_test)
            if len(frames)>=window:
                # Make a prediction by using the model
                test_input = np.expand_dims(np.array(frames[-window:]),axis=0)
                print(test_input.shape)
                y_predicted = session.run(output, feed_dict={gru.input_layer: test_input})
                y_predicted_confidence = np.max(y_predicted)
                y_predicted = np.argmax(y_predicted,axis=-1)
                action = le.inverse_transform(y_predicted)
                print('\nPredictions: \n',y_predicted,"\nConfidence: ",y_predicted_confidence)
                print("Inference Time: ",time.time()-start)
                # Draw landmarks
                draw_styled_landmarks(image, results)
                cv2.putText(image, 'Frame Number {}'.format(count), (15,45), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(image, 'Action: {} '.format(action), (image.shape[0]//2,45), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 1, cv2.LINE_AA)
                clear_output(wait=True)

                # Show to screen
                cv2.imshow('frame',image)
                time.sleep(0.01)
                frames = frames[-(window-1):]
            else:
                continue
        else: 
            break
        # Break gracefully
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
draw_landmarks(frame, results)

In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


In [ ]:
# 3. Extract Keypoint Values

In [ ]:
#len(results.left_hand_landmarks.landmark)

In [ ]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)


In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)


In [ ]:
#face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() 
    #if results.face_landmarks 
   #else np.zeros(1404)


In [ ]:
result_test = extract_keypoints(results)


In [ ]:
result_test.shape

In [ ]:
result_test=result_test.reshape(1,result_test.shape[0],1)
result_test.shape


In [ ]:
# Make a prediction by using the model
import pickle 
le = pickle.load(open('C:/Users/Lab/Desktop/Barathi/ActionDetectionforSignLanguage/GRU modified/GRU/labelencoder.pkl','rb'))
y_predicted = session.run(output, feed_dict={gru.input_layer: result_test})
y_predicted_confidence = np.max(y_predicted)
y_predicted = np.argmax(y_predicted,axis=-1)
print(y_predicted)

print('\nPredictions: \n',le.inverse_transform(y_predicted),"\nConfidence: ",y_predicted_confidence)


In [ ]:
np.save('0', result_test)

# 4. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello','thankyou','takecare','good','fail','like','friend','think','phonecall','meet','you','iloveyou','location'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
#start_folder = 30

In [ ]:
for action in actions: 
   # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)
                print(results)
                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                cv2.imshow('opencv feed' ,image)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
label_map

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

In [ ]:
X = np.array(sequences)

In [ ]:
X.shape

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [ ]:
y_test.shape

In [ ]:
X.shape

# 7. Build and Train LSTM Neural Network

# Standard GRU

# LSTM

# SimpleRNN

# Proposed GRU

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('LogsMGRU')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(GRU(128, return_sequences=True, activation='elu', input_shape=(30,1662)))
model.add(Dropout(0.2))
model.add(GRU(64, return_sequences=True, activation='tanh'))
model.add(Dropout(0.3))
model.add(GRU(32, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
history_4 = model.fit(X_train, y_train, epochs=100, validation_split=0.2,callbacks=[tb_callback])

In [ ]:
history_4.history

In [ ]:
model.summary()

In [ ]:
acc = history_4.history['categorical_accuracy']
val_acc = history_4.history['val_categorical_accuracy']

loss = history_4.history['loss']
val_loss = history_4.history['val_loss']

plt.figure(figsize=(15, 15))
plt.subplot(2, 1, 2)
plt.plot(acc,'-b*', label='Training Accuracy')
plt.plot(val_acc,'-r*', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 1)
plt.plot(loss,'-b*', label='Training Loss')
plt.plot(val_loss,'-r*', label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.5])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()




In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[1])] 

In [ ]:
actions[np.argmax(y_test[1])]

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,classification_report

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
from sklearn import metrics

In [ ]:
print(f"Classification report for Sign Gestures \n"
      f"{metrics.classification_report(ytrue, yhat)}\n")

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(ytrue, yhat)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(ytrue, yhat)

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(ytrue, yhat)

In [ ]:
print("Evaluate on test data")
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss, test acc:", results)

# Bidirectional GRU

# Bidirectional LSTM